<a href="https://colab.research.google.com/github/DAVIDnHANG/DS-Unit-2-Regression-Classification/blob/master/module3/A3_Regression_%26_Classification_SelectKBest%2C_Ridge_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pca cares about the magnitude of things.

#why should we scalize?
1) standardize first.
2) to make slopes for different columns comparable
3)

correlation matrix - 

```
.5, .7 bigger than .5. then this is good.

```



In [0]:
#imports here
import pandas as pd
import pandas_profiling
import datetime as dt
import numpy as np

In [0]:
# Read New York City property sales data
df = pd.read_csv('https://raw.githubusercontent.com/DAVIDnHANG/DS-Unit-2-Regression-Classification/master/data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int))


In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
#restrict the data set between 100k to 2million of SALE_PRICE

#Turn the SALE PRICE, a object str, into a int.
df['SALE_PRICE'].astype(str).astype(int)
df['SALE_PRICE'] =df['SALE_PRICE'].replace(np.nan,0)
df=df.loc[(df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000)]
df.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
65,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,691,1130,NaN,R4,"505 WEST 19TH STREET, 7D",7D,10011.0,1.0,0.0,1.0,"18,975",1077.0,2013.0,2,R4,2000000,01/02/2019


In [0]:
#restrict the date set further into only '01 ONE FAMILY DWELLINGS'
df=df.loc[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')]
df.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019


In [0]:
#SALE_DATE is in the format DD/MM/YYYY
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
start_date = '01/01/2019'
end_date = '03/30/2019'
mask = (df['SALE_DATE'] > start_date) & (df['SALE_DATE'] <= end_date)
train_Date_March_April = df.loc[mask]
print(train_Date_March_April.shape)

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
start_date_test = '04/01/2019'
end_date_test = '04/30/2019'
mask_test = (df['SALE_DATE'] > start_date_test) & (df['SALE_DATE'] <= end_date_test)
test_April = df.loc[mask_test]
print(test_April.shape)


target = 'SALE_PRICE'

y_train = train_Date_March_April[target]
y_test = test_April[target]

(2515, 21)
(606, 21)


In [0]:
train_Date_March_April.columns

BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                float64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
APARTMENT_NUMBER                          object
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                          object
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE           

In [0]:
train_Date_March_April.columns

In [0]:
drop =['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'EASE-MENT', 'BLOCK', 'LOT', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_DATE']
train_Date_March_April = train_Date_March_April.drop(drop, axis=1)    
test_April = test_April.drop(drop, axis=1)  
train_Date_March_April.dtypes 

ZIP_CODE      float64
YEAR_BUILT    float64
SALE_PRICE      int64
dtype: object

KbestFit with scaler

In [0]:
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_Date_March_April)
X_test_scaled = scaler.transform(test_April)
for k in range(1, len(train_Date_March_April.columns)+1):
   print(f'{k} features')
   selector = SelectKBest(score_func=f_regression, k=k)
   X_train_selected = selector.fit_transform(X_train_scaled, y_train)
   X_test_selected = selector.transform(X_test_scaled)
   model = LinearRegression()
   model.fit(X_train_selected, y_train)
   y_pred = model.predict(X_test_selected)
   mae = mean_absolute_error(y_test, y_pred)
   print(f'Test MAE: ${mae:,.0f} \n')

#make sure you feed only numeric datatypes

1 features
Test MAE: $214,196 

2 features
Test MAE: $213,832 

3 features
Test MAE: $0 



In [0]:
selector = SelectKBest(score_func=f_regression, k=15)
all_names = train_Date_March_April.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_name[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)
    
print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

# Ridge *Regression*

In [0]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import scale


#SALE_DATE is in the format DD/MM/YYYY
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
start_date = '01/01/2019'
end_date = '03/30/2019'
mask = (df['SALE_DATE'] > start_date) & (df['SALE_DATE'] <= end_date)
train_Date_March_April = df.loc[mask]
print(train_Date_March_April.shape)

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
start_date_test = '04/01/2019'
end_date_test = '04/30/2019'
mask_test = (df['SALE_DATE'] > start_date_test) & (df['SALE_DATE'] <= end_date_test)
test_April = df.loc[mask_test]
print(test_April.shape)

target = 'SALE_PRICE'
high_cardiantlity = ['BOROUGH','NEIGHBORHOOD','BUILDING_CLASS_CATEGORY',
       'BLOCK','LOT' ,'LOT', 'EASE-MENT','BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT',
       'ADDRESS','ADDRESS','APARTMENT_NUMBER' ,'APARTMENT_NUMBER', 'ZIP_CODE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'ZIP_CODE', 'GROSS_SQUARE_FEET']
features = train_Date_March_April.columns.drop(high_cardiantlity + [target])

target = 'SALE_PRICE'

X_train = train_Date_March_April[features]
y_test = train_Date_March_April[target]

X_test = test_April[features]
y_test = test_April[target]


In [0]:
# Let's try good old least squares!
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = df.drop('Price', axis='columns')
y = df.Price

lin_reg = LinearRegression().fit(X, y)
mean_squared_error(y, lin_reg.predict(X))

21.894831181729206

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43)
lin_reg_split = LinearRegression().fit(X_train, y_train)
print(mean_squared_error(y, lin_reg_split.predict(X)))
print(mean_squared_error(y_test, lin_reg_split.predict(X_test)))

22.347018673376052
26.273991426429014


In [0]:
# Now with regularization via ridge regression
from sklearn.linear_model import Ridge

ridge_reg = Ridge().fit(X, y)
mean_squared_error(y, ridge_reg.predict(X))

21.895862166800143

In [0]:
X.dtypes

CRIM       float64
ZN         float64
INDUS      float64
CHAS       float64
NOX        float64
RM         float64
AGE        float64
DIS        float64
RAD        float64
TAX        float64
PTRATIO    float64
B          float64
LSTAT      float64
dtype: object

In [0]:
ridge_reg_split = Ridge().fit(X_train, y_train)
mean_squared_error(y_test, ridge_reg_split.predict(X_test))

26.192201358877668

In [0]:
alphas = []
mses = []

for alpha in range(0, 200, 1):
    ridge_reg_split = Ridge(alpha=alpha).fit(X_train, y_train)
    mse = mean_squared_error(y_test, ridge_reg_split.predict(X_test))
    print(alpha, mse)
    alphas.append(alpha)
    mses.append(mse)

0 26.273991426429053
1 26.192201358877668
2 26.118328007697226
3 26.051117952293595
4 25.989569283205444
5 25.93287356811407
6 25.880372753122625
7 25.831526788692837
8 25.785889053385123
9 25.743087513207442
10 25.702810145277628
11 25.66479356379559
12 25.628814073392263
13 25.59468057863174
14 25.56222892458644
15 25.53131734932431
16 25.50182280665663
17 25.473637974726547
18 25.44666880864133
19 25.420832527348388
20 25.396055949160893
21 25.372274108781504
22 25.349429102822004
23 25.327469121742823
24 25.30634763462536
25 25.286022699825878
26 25.266456379775086
27 25.24761424230921
28 25.229464934192976
29 25.211979815108453
30 25.19513264248022
31 25.178899299197408
32 25.163257557659424
33 25.14818687468406
34 25.13366821272317
35 25.11968388357409
36 25.106217411385522
37 25.093253412260974
38 25.080777488180427
39 25.068776133307583
40 25.057236651039766
41 25.046147080399017
42 25.035496130566347
43 25.02527312253186
44 25.015467936977434
45 25.006070967630855
46 24.997073

In [0]:
df_tiny = df.sample(10, random_state=27)
print(df_tiny.shape)
X = df_tiny.drop('Price', axis='columns')
y = df_tiny.Price

lin_reg = LinearRegression().fit(X, y)
lin_reg.score(X, y)  # Perfect multi-collinearity!
# NOTE - True OLS would 💥 here
# scikit protects us from actual error, but still gives a poor model

(10, 14)


1.0